In [2]:
import numpy as np
from scipy import linalg
from scipy.optimize import minimize

In [3]:
#                 (0,244)--------(294,244)
#                     |               |
# Polygone ->         |               |
#                     |               |
#                   (0,0)--------(294, 0)

Zhang’s Calibration DLT METHOD

In [5]:
pol1 = (554/1280, 237/720)
pol2 = (343/1280, 490/720)
pol3 = (934/1280, 693/720)
pol4 = (978/1280, 278/720)
pol1_2 = (461/1280, 349/720)
pol2_2 = (583/1280, 572/720)
pol3_2 = (961/1280, 438/720)
pol4_2 = (727/1280, 254/720)

gps_points = [(54.725242, 55.940438),#0
              (54.725207, 55.940667),#1
              (54.725185, 55.940805),#2

              (54.725309, 55.940874),#3
              (54.725433, 55.940939),#4
              (54.725458, 55.940803),#5

              (54.725500, 55.940560),#6
              (54.725367, 55.940505)]#7

x_res, y_res = (1920, 1080)# resolution
polygon = np.array([(pol1[0] * x_res, pol1[1] * y_res), 
                    (pol1_2[0] * x_res, pol1_2[1] * y_res), 
                    (pol2[0] * x_res, pol2[1] * y_res),
                    (pol2_2[0] * x_res, pol2_2[1] * y_res),
                    (pol3[0] * x_res, pol3[1] * y_res),
                    (pol3_2[0] * x_res, pol3_2[1] * y_res),
                    (pol4[0] * x_res, pol4[1] * y_res), 
                    (pol4_2[0] * x_res, pol4_2[1] * y_res)], dtype=int)#points for polygone in the center

def zang(pt):
    
    # pointsW = [(0, 244), (0,122), (0, 0), (147, 0), (294, 0), (294, 122), (294, 244), (147, 244)]
    pointsW = gps_points
    pointsPX = polygon
    print(pointsW, '\n', pointsPX)

    if len(pointsPX) != len(pointsW): return

    A = np.zeros((len(pointsW)*2, 9))
    for k, W, PX in zip(range(0, len(pointsW)*2, 2),pointsW, pointsPX):
        xw = W[0]
        yw = W[1]

        u = PX[0]
        v = PX[1]

        A[k] = [-xw, -yw, -1, 0, 0, 0, u*xw, u*yw, u]
        A[k+1] = [0, 0, 0, -xw, -yw, -1, v*xw, v*yw, v]
    
    U,D,V = np.linalg.svd(A)
    m_ext = np.reshape(V[-1] / V[-1][-1], (3,3))
    vector = np.array([pt[0], pt[1], 1])
    print(m_ext)
    #BACKWARD PIXEL -> WORLD
    result = np.linalg.solve(m_ext, vector)
    result = result / result[-1]
    print(result[:2])

    #FORWARD WORLD -> PIXEL
    # result = np.dot(m_ext, vector)
    # result = result / result[-1]
    # print(result)

zang([1146, 435])

[(54.725242, 55.940438), (54.725207, 55.940667), (54.725185, 55.940805), (54.725309, 55.940874), (54.725433, 55.940939), (54.725458, 55.940803), (54.7255, 55.94056), (54.725367, 55.940505)] 
 [[ 831  355]
 [ 691  523]
 [ 514  735]
 [ 874  858]
 [1401 1039]
 [1441  657]
 [1467  417]
 [1090  381]]
[[ 6.09366629e+01 -2.34309430e+01 -2.02401642e+03]
 [ 1.17909255e+00  1.10536131e+01 -6.82862154e+02]
 [ 4.02084845e-03 -2.18092262e-02  1.00000000e+00]]
[54.72537655 55.94056891]
